In [27]:
import requests
import json
import pandas as pd
import datetime
import s3_file_operations as s3_ops

In [30]:
url = "https://rickandmortyapi.com/api/character"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"info":{"count":826,"pages":42,"next":"https://rickandmortyapi.com/api/character?page=2","prev":null},"results":[{"id":1,"name":"Rick Sanchez","status":"Alive","species":"Human","type":"","gender":"Male","origin":{"name":"Earth (C-137)","url":"https://rickandmortyapi.com/api/location/1"},"location":{"name":"Citadel of Ricks","url":"https://rickandmortyapi.com/api/location/3"},"image":"https://rickandmortyapi.com/api/character/avatar/1.jpeg","episode":["https://rickandmortyapi.com/api/episode/1","https://rickandmortyapi.com/api/episode/2","https://rickandmortyapi.com/api/episode/3","https://rickandmortyapi.com/api/episode/4","https://rickandmortyapi.com/api/episode/5","https://rickandmortyapi.com/api/episode/6","https://rickandmortyapi.com/api/episode/7","https://rickandmortyapi.com/api/episode/8","https://rickandmortyapi.com/api/episode/9","https://rickandmortyapi.com/api/episode/10","https://rickandmortyapi.com/api/episode/11","https://rickandmortyapi.com/api/episode/12","https://ric

In [4]:
response.json()

{'info': {'count': 826,
  'pages': 42,
  'next': 'https://rickandmortyapi.com/api/character?page=2',
  'prev': None},
 'results': [{'id': 1,
   'name': 'Rick Sanchez',
   'status': 'Alive',
   'species': 'Human',
   'type': '',
   'gender': 'Male',
   'origin': {'name': 'Earth (C-137)',
    'url': 'https://rickandmortyapi.com/api/location/1'},
   'location': {'name': 'Citadel of Ricks',
    'url': 'https://rickandmortyapi.com/api/location/3'},
   'image': 'https://rickandmortyapi.com/api/character/avatar/1.jpeg',
   'episode': ['https://rickandmortyapi.com/api/episode/1',
    'https://rickandmortyapi.com/api/episode/2',
    'https://rickandmortyapi.com/api/episode/3',
    'https://rickandmortyapi.com/api/episode/4',
    'https://rickandmortyapi.com/api/episode/5',
    'https://rickandmortyapi.com/api/episode/6',
    'https://rickandmortyapi.com/api/episode/7',
    'https://rickandmortyapi.com/api/episode/8',
    'https://rickandmortyapi.com/api/episode/9',
    'https://rickandmortyapi.

we see there are 42 pages having Charcter data

- but first lets create a dataframe to stire the characters returned by the first page of response

In [31]:
first_page_df = pd.DataFrame(response.json().get('results', []))

print(first_page_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        20 non-null     int64 
 1   name      20 non-null     object
 2   status    20 non-null     object
 3   species   20 non-null     object
 4   type      20 non-null     object
 5   gender    20 non-null     object
 6   origin    20 non-null     object
 7   location  20 non-null     object
 8   image     20 non-null     object
 9   episode   20 non-null     object
 10  url       20 non-null     object
 11  created   20 non-null     object
dtypes: int64(1), object(11)
memory usage: 2.0+ KB
None


In [32]:
first_page_df.head()

,id,name,status,species,type,gender,origin,location,image,episode,url,created
0,1,Rick Sanchez,Alive,Human,,Male,"{'name': 'Earth (C-137)', 'url': 'https://rick...","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,"{'name': 'unknown', 'url': ''}","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z
4,5,Jerry Smith,Alive,Human,,Male,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z


In [33]:

page = 1
next = True

all_data = []

while next:
    print(f"Extracting page {page} Data.....")

    response = requests.get(f"https://rickandmortyapi.com/api/character?page={str(page)}")

    data = response.json().get('results', [])

    all_data.extend(data)

    # checking if the value of next url is null for the loop to stop
    if response.json().get('info', {}).get("next") is not None: 
        page += 1
    else:
        break


characters_df = pd.DataFrame(all_data)

print(characters_df.shape)





Extracting page 1 Data.....
Extracting page 2 Data.....
Extracting page 3 Data.....
Extracting page 4 Data.....
Extracting page 5 Data.....
Extracting page 6 Data.....
Extracting page 7 Data.....
Extracting page 8 Data.....
Extracting page 9 Data.....
Extracting page 10 Data.....
Extracting page 11 Data.....
Extracting page 12 Data.....
Extracting page 13 Data.....
Extracting page 14 Data.....
Extracting page 15 Data.....
Extracting page 16 Data.....
Extracting page 17 Data.....
Extracting page 18 Data.....
Extracting page 19 Data.....
Extracting page 20 Data.....
Extracting page 21 Data.....
Extracting page 22 Data.....
Extracting page 23 Data.....
Extracting page 24 Data.....
Extracting page 25 Data.....
Extracting page 26 Data.....
Extracting page 27 Data.....
Extracting page 28 Data.....
Extracting page 29 Data.....
Extracting page 30 Data.....
Extracting page 31 Data.....
Extracting page 32 Data.....
Extracting page 33 Data.....
Extracting page 34 Data.....
Extracting page 35 Data

In [34]:
characters_df.head()

,id,name,status,species,type,gender,origin,location,image,episode,url,created
0,1,Rick Sanchez,Alive,Human,,Male,"{'name': 'Earth (C-137)', 'url': 'https://rick...","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z
1,2,Morty Smith,Alive,Human,,Male,"{'name': 'unknown', 'url': ''}","{'name': 'Citadel of Ricks', 'url': 'https://r...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/1, ht...",https://rickandmortyapi.com/api/character/2,2017-11-04T18:50:21.651Z
2,3,Summer Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/3,2017-11-04T19:09:56.428Z
3,4,Beth Smith,Alive,Human,,Female,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/4,2017-11-04T19:22:43.665Z
4,5,Jerry Smith,Alive,Human,,Male,"{'name': 'Earth (Replacement Dimension)', 'url...","{'name': 'Earth (Replacement Dimension)', 'url...",https://rickandmortyapi.com/api/character/avat...,"[https://rickandmortyapi.com/api/episode/6, ht...",https://rickandmortyapi.com/api/character/5,2017-11-04T19:26:56.301Z


In [35]:
# Loading the extracted data to AWS S3
bucket = "de-masterclass"


# Write exchanges_data to S3
s3_ops.write_data_to_s3(characters_df,
                        bucket_name=bucket,
                        key=f"Rick&Morty/Untransformed/Character.csv")

Data saved to S3 with bucket_name: de-masterclass and key: Rick&Morty/Character/2024/08/10/13/26/Character.csv


True